# Objet : Test pandas

## Objectif

- list vers pandas series

## Résultats
- à préciser

## Usages possibles 
- rempacement _keys et _keys par Series

## Autres points
- à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

In [39]:
import json
import shapely
from shapely.geometry import Point, Polygon
from json_ntv import ShapelyConnec

data =pd.Series([Point(1,2), Point(1,3), Point(2,4), Polygon([[1.0, 2.0], [1.0, 3.0], [2.0, 4.0]]),
Polygon([[1.0, 2.0], [1.0, 30.0], [30.0, 30.0], [30,2]], [[[5.0, 16.0], [5.0, 27.0], [20.0, 27.0]]])])

coor = data.apply(ShapelyConnec.to_coord)
print(coor)

coor2 = data.apply(shapely.geometry.mapping)
print(coor2)

coor3 = coor2.apply(json.dumps)
print(coor3)

coor4 = coor3.apply(json.loads)
print(coor4)

0                                           [1.0, 2.0]
1                                           [1.0, 3.0]
2                                           [2.0, 4.0]
3    [[[1.0, 2.0], [1.0, 3.0], [2.0, 4.0], [1.0, 2....
4    [[[1.0, 2.0], [1.0, 30.0], [30.0, 30.0], [30.0...
dtype: object
0         {'type': 'Point', 'coordinates': (1.0, 2.0)}
1         {'type': 'Point', 'coordinates': (1.0, 3.0)}
2         {'type': 'Point', 'coordinates': (2.0, 4.0)}
3    {'type': 'Polygon', 'coordinates': (((1.0, 2.0...
4    {'type': 'Polygon', 'coordinates': (((1.0, 2.0...
dtype: object
0         {"type": "Point", "coordinates": [1.0, 2.0]}
1         {"type": "Point", "coordinates": [1.0, 3.0]}
2         {"type": "Point", "coordinates": [2.0, 4.0]}
3    {"type": "Polygon", "coordinates": [[[1.0, 2.0...
4    {"type": "Polygon", "coordinates": [[[1.0, 2.0...
dtype: object
0         {'type': 'Point', 'coordinates': [1.0, 2.0]}
1         {'type': 'Point', 'coordinates': [1.0, 3.0]}
2         {'type': 'Poi

### tests styler

In [28]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
styler = df.style.highlight_max()

styler

,A,B
0,1,4
1,2,5
2,3,6


In [30]:
styler.to_html()

'<style type="text/css">\n#T_04ac4_row2_col0, #T_04ac4_row2_col1 {\n  background-color: yellow;\n}\n</style>\n<table id="T_04ac4">\n  <thead>\n    <tr>\n      <th class="blank level0" >&nbsp;</th>\n      <th id="T_04ac4_level0_col0" class="col_heading level0 col0" >A</th>\n      <th id="T_04ac4_level0_col1" class="col_heading level0 col1" >B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th id="T_04ac4_level0_row0" class="row_heading level0 row0" >0</th>\n      <td id="T_04ac4_row0_col0" class="data row0 col0" >1</td>\n      <td id="T_04ac4_row0_col1" class="data row0 col1" >4</td>\n    </tr>\n    <tr>\n      <th id="T_04ac4_level0_row1" class="row_heading level0 row1" >1</th>\n      <td id="T_04ac4_row1_col0" class="data row1 col0" >2</td>\n      <td id="T_04ac4_row1_col1" class="data row1 col1" >5</td>\n    </tr>\n    <tr>\n      <th id="T_04ac4_level0_row2" class="row_heading level0 row2" >2</th>\n      <td id="T_04ac4_row2_col0" class="data row2 col0" >3</td>\n      <td i

In [31]:
styler.to_json()

AttributeError: 'Styler' object has no attribute 'to_json'

### tests numpy

In [138]:
import numpy as np
from time import time

coef = 40
period = 20
repeat = 50
leng = coef * period * repeat
lseq = coef * period
tn = 0
tp = 0
for i in range(50):
    t0 =time()
    seq = np.stack(tuple([np.arange(period)] * coef), axis=1).reshape(lseq)
    keys1 = list(np.stack(tuple([seq] * repeat)).reshape(leng))
    tn += time()-t0

    t0 =time()
    keys2 = [(i % lseq) // coef for i in range(leng)]
    tp += time()-t0
print(tn/10, tp/10, tp/tn, keys1 == keys2)

0.014781665802001954 0.05381264686584473 3.640499493538585 True


------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [2]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
#os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from observation import Ilist, Iindex
from copy import copy

chemin='C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/Environnemental-Sensing/python/validation/air/data_lcsqa/'

In [20]:
import pandas as pd
from datetime import date
import json

In [27]:
sr = pd.Series([date(2021,1,1), date(2022, 1, 2)])
print(sr)
sr2 = sr.astype(str) # conversion date -> str
js = sr2.to_json(orient='records', date_format='iso', default_handler=str)
print(js)

sr3 = pd.read_json(js, typ='series')
sr4 = pd.to_datetime(sr3).dt.date # conversion str -> date
print(sr4.equals(sr))
print(sr4)


0    2021-01-01
1    2022-01-02
dtype: object
["2021-01-01","2022-01-02"]
True
0    2021-01-01
1    2022-01-02
dtype: object


In [3]:
data = []
t0 = time()
annee = 2022
mois = 1
jour = 1
for i in range(4):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2', len(data2), list(data2), time()-t0)

data2 197455 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 1.7311534881591797


In [4]:
t0=time()
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.delindex('valeur brute')
idxs2.setvar('valeur')

idxs (len, lenlidx, sumcodec) :  197455 22 4462 1.7334957122802734


In [15]:
#calcul de values à partir de keys et codec
t0=time()
for idx in idxs2.lindex:
    val = idx.values
print('list to val old: ', time()-t0)
#génération d'une liste de Series à partir d'une liste de codec
t0=time()
pdcodec=[pd.Series(idx.codec) for idx in idxs2.lindex]
print('list to codec : ', time()-t0)#génération d'une liste de Series à partir d'une liste de keys
#génération d'une liste de Series simple à partir d'une liste de keys
t0=time()
pdkeyss=[pd.Series(idx.keys, dtype='int64') for idx in idxs2.lindex]
print('list to cat simple : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
t0=time()
pdkeys=[pd.Series(idx.keys, dtype='category') for idx in idxs2.lindex]
print('list to cat : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
pdkeys=[]
t0=time()
for idx in idxs2.lindex:
    pdkeys.append(pd.Series(idx.keys, dtype='category'))
print('list to cat : ', time()-t0)#calcul de values à partir de keys et codec avec une Series categorical (default)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys, dtype='category').cat.rename_categories(idx.codec))
print('list to val (def): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series map (all)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys).map(pd.Series(idx.codec)))
print('list to val (all map): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(pd.DataFrame(keys.cat.rename_categories(pd.Series(zip(idx.codec, range(len(idx.codec))))).tolist())[0])
print('list to val (all tuple): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec (default)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.cat.rename_categories(idx.codec))
print('cat to val (def): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.map(pd.Series(idx.codec)))
print('cat to val (all): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeyss):
    val = list(keys.map(cod))
print('simple to val (all series simple): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeys):
    val = list(keys.map(cod))
print('cat to val (all series): ', time()-t0)

list to val old:  0.658961296081543
list to codec :  0.008095264434814453
list to cat simple :  0.5761041641235352
list to cat :  1.3805665969848633
list to cat :  1.4476771354675293
list to val (def):  1.520902156829834
list to val (all map):  1.7561311721801758
list to val (all tuple):  2.2119081020355225
cat to val (def):  0.17286372184753418
cat to val (all):  0.20073533058166504
simple to val (all series simple):  0.5166947841644287
cat to val (all series):  0.18467140197753906


In [7]:
#print(val[:100])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1]


In [23]:
#ESValue
from observation import DatationValue, NamedValue
ser = pd.Series([DatationValue('date'), DatationValue('date2'), DatationValue('date'), NamedValue(3,'test')], dtype='category')
print(ser.cat.categories, '\n', list(ser.cat.codes))
ser2 = pd.Series([0,11,0,22]).astype('category')
serbis = ser2.cat.rename_categories([DatationValue('date'), DatationValue('date2'),  NamedValue(3,'test')])
min(ser.astype('object') == serbis.astype('object'))

Index(["date", {"test": 3}, "date2"], dtype='object') 
 [0, 2, 0, 1]


True

In [34]:
#generation dataframe à partir de Ilist
ilm = Ilist.obj([['plants', ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']],
                  ['quantity', ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']],
                  ['product', ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']],
                  ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]])
df = pd.concat([idx.to_pandas(index=False, series=False).astype('category') for idx in ilm.lindex], axis=1)
col = pd.MultiIndex.from_arrays([ilm.lname, ilm.lisvar], names=['name', 'var'])
df.columns = col
print(df)

name     plants quantity  product price
var       False    False    False  True
0         fruit       kg    apple   1.0
1         fruit    10 kg    apple  10.0
2         fruit       kg   orange   2.0
3         fruit    10 kg   orange  20.0
4     vegetable       kg  peppers   1.5
5     vegetable    10 kg  peppers  15.0
6     vegetable       kg   banana   0.5
7         fruit    10 kg   banana   5.0


In [44]:
# fonction append
s2 = pd.Series(['fruit', 'kg', 'banana', 7], index=df.columns)
df = pd.concat([df, s2.to_frame().T], ignore_index=True).astype('category') 
ds = df['price'].squeeze()
ds.cat.categories

Float64Index([0.5, 1.0, 1.5, 2.0, 5.0, 7.0, 10.0, 15.0, 20.0], dtype='float64')

In [50]:
a= pd.Series(['b', 'a', 'a', 'd']).astype('category')
b = pd.Series([1,2,4,3]).astype('category')
coup = pd.Series(list(zip(*(list(a), list(b))))).astype('category')
print(coup)

print(a.cat.codes)
print(coup.cat.codes)
res = coup.cat.codes == a.cat.codes
print(res)

0    (b, 1)
1    (a, 2)
2    (a, 4)
3    (d, 3)
dtype: category
Categories (4, object): [('a', 2), ('a', 4), ('b', 1), ('d', 3)]
0    1
1    0
2    0
3    2
dtype: int8
0    2
1    0
2    1
3    3
dtype: int8
0    False
1     True
2    False
3    False
dtype: bool


In [51]:
a = pd.Series([0,1,2,3,2]).astype('category')
codec = ['a', 'b', 'c', 'b']
print(a.cat.codes.map(pd.Series(codec)))

0    a
1    b
2    c
3    b
4    c
dtype: object


In [15]:
#filtrage suivant var
df.loc[:,(slice(None),False)]

name,plants,quantity,product
var,False,False,False
0,fruit,kg,apple
1,fruit,10 kg,apple
2,fruit,kg,orange
3,fruit,10 kg,orange
4,vegetable,kg,peppers
5,vegetable,10 kg,peppers
6,vegetable,kg,banana
7,fruit,10 kg,banana


In [6]:
full = [[name, list(ds)] for name,ds in df.items()]
print(full, '\n')
default = [[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in df.astype('category').items()]
print(default)

[[(False, 'plants'), ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']], [(False, 'quantity'), ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']], [(False, 'product'), ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']], [(True, 'price'), [1.0, 10.0, 2.0, 20.0, 1.5, 15.0, 0.5, 5.0]]] 

[[(False, 'plants'), ['fruit', 'vegetable'], [0, 0, 0, 0, 1, 1, 1, 0]], [(False, 'quantity'), ['10 kg', 'kg'], [1, 0, 1, 0, 1, 0, 1, 0]], [(False, 'product'), ['apple', 'banana', 'orange', 'peppers'], [0, 0, 2, 2, 3, 3, 1, 1]], [(True, 'price'), [0.5, 1.0, 1.5, 2.0, 5.0, 10.0, 15.0, 20.0], [1, 5, 3, 7, 2, 6, 0, 4]]]


In [15]:
d = {"one": [1.0, 2.0, 3.0, 4.0], "two": [4.0, 3.0, 2.0, 1.0]}
df = pd.DataFrame(d)
df["three"] = True
df["four"] = [True, True, False, False]
df["total"] = True
df['somme'] = 0
f =[1,3]
df.loc[f, "three"] = False
df['total'] = df['total'] & df['three']
df['total'] = df['total'] & df['four']
df['somme'] = df['somme'] - df['three'] +1
df['somme'] = df['somme'] - df['four'] +1
print(df)
print(df.four.sum(), df.total.sum())
maxi = max(df.somme) 
df_max = df[df.somme == maxi]
print(maxi, len(df_max))
print(df_max.loc[:,('one','three', 'four', 'total')])

   one  two  three   four  total  somme
0  1.0  4.0   True   True   True      0
1  2.0  3.0  False   True  False      1
2  3.0  2.0   True  False  False      1
3  4.0  1.0  False  False  False      2
2 1
2 1
   one  three   four  total
3  4.0  False  False  False


In [7]:
print(ilm.to_obj(modecodec='full'), '\n')
print(ilm.to_obj(modecodec='default'))

[['plants', ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']], ['quantity', ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']], ['product', ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']], ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]] 

[['plants', ['fruit', 'vegetable'], [0, 0, 0, 0, 1, 1, 1, 0]], ['quantity', ['kg', '10 kg'], [0, 1, 0, 1, 0, 1, 0, 1]], ['product', ['orange', 'apple', 'peppers', 'banana'], [1, 1, 0, 0, 2, 2, 3, 3]], ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]]


In [8]:
#generation de json (full et default)
import json
t0=time()
full = json.dumps([[name, list(ds)] for name,ds in data2.items()])
print('fullsize', len(full), time()-t0)
t0=time()
default = json.dumps([[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in data2.astype('category').items()])
print('defaultsize', len(default), time()-t0)


fullsize 87068691 1.397794246673584
defaultsize 24003010 1.4524955749511719


In [9]:
t0=time()
fullsize = len(idxs2.to_obj(encoded=True, modecodec='full'))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs2.to_obj(encoded=True, modecodec='nokeys'))
print('minsize', minsize, time()-t0)

fullsize 86202592 15.800340414047241
minsize 1694137 0.6880073547363281


----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque).

In [10]:
t0=time()
defaultsize = len(idxs2.to_obj(encoded=True, modecodec='default'))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs2.indicator(fullsize, defaultsize))
pprint(idxs2.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl']), width=120)
pprint(idxs2.indexinfos(keys=['linkrate']))

defaultsize 22942572 1.483217477798462
indicator default :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 3.523, 'unicity level': 0.0, 'object lightness': 0.266, 'gain': 0.734}
[{'lencodec': 144, 'name': 'Date de début', 'num': 0, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 144, 'name': 'Date de fin', 'num': 1, 'parent': 0, 'typecoupl': 'coupled'},
 {'lencodec': 18, 'name': 'Organisme', 'num': 2, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'code zas', 'num': 3, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'Zas', 'num': 4, 'parent': 3, 'typecoupl': 'coupled'},
 {'lencodec': 533, 'name': 'code site', 'num': 5, 'parent': 5, 'typecoupl': 'crossed'},
 {'lencodec': 533, 'name': 'nom site', 'num': 6, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 5, 'name': "type d'implantation", 'num': 7, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 9, 'name': 'Polluant', 'num': 8, 'parent': 13, 'typecoupl': '

----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [11]:
#idxs.setcanonorder().sort()
t0=time()
optimizesize = len(idxs2.to_obj(modecodec='optimize', encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize))
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='optimize', format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

optimizesize  13139380 2.0916759967803955
indicator optimize :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 2.016, 'unicity level': 0.0, 'object lightness': 0.152, 'gain': 0.848}
cborsize 6852732 1.8763666152954102
indicator cbor :  {'init values': 6507754, 'mean size': 13.246, 'unique values': 1654, 'mean coding size': 1.05, 'unicity level': 0.0, 'object lightness': 0.079, 'gain': 0.921}


----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [12]:
t0=time()
idxs3 = Ilist.from_obj(js)
print('fromcbor', len(idxs3), time()-t0)
t0=time()
verif = idxs2 == idxs3
print('controle égalité :', verif, time()-t0)


fromcbor 295807 4.430227041244507
controle égalité : True 2.1935391426086426
